In [10]:
import logging
from copy import copy
from enum import Enum
import matplotlib.pyplot as plt
import cv2
import scipy

import numpy as np
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph, AngleAxis,
                         Concatenate, DiagramBuilder, InputPortIndex,
                         LeafSystem, MeshcatVisualizer, Parser,
                         PiecewisePolynomial, PiecewisePose, PointCloud,
                         PortSwitch, RandomGenerator, RigidTransform,
                         RollPitchYaw, Simulator, StartMeshcat,
                         UniformlyRandomRotationMatrix, FindResourceOrThrow, Sphere,
                         RotationMatrix, SpatialVelocity, InverseDynamicsController,
                         MultibodyPlant, PassThrough, Demultiplexer, Adder, StateInterpolatorWithDiscreteDerivative,
                         MathematicalProgram, Solve, PointCloud, ConstantVectorSource, Integrator, DiscreteDerivative,
                         DifferentialInverseKinematicsParameters, DifferentialInverseKinematicsIntegrator,
                         InverseKinematics, MeshcatPointCloudVisualizer, Rgba, DiscreteContactSolver,
                         PiecewisePose, PiecewisePolynomial, PiecewiseQuaternionSlerp, AngleAxis)

from manipulation import FindResource, running_as_notebook
from manipulation.clutter import GenerateAntipodalGraspCandidate
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.pick import (MakeGripperCommandTrajectory, MakeGripperFrames,
                               MakeGripperPoseTrajectory)
from manipulation.scenarios import (AddIiwaDifferentialIK, AddPackagePaths, AddRgbdSensors,
                                    MakeManipulationStation, ycb, AddShape, AddIiwa)
from manipulation.utils import AddPackagePaths, FindResource

class NoDiffIKWarnings(logging.Filter):
    def filter(self, record):
        return not record.getMessage().startswith('Differential IK')

logging.getLogger("drake").addFilter(NoDiffIKWarnings())

TIME_STEP = 0.05


In [2]:
def AddBin(plant,
           iiwa_model_instance):
    parser = Parser(plant)
    gripper = parser.AddModelFromFile(
                "../manipulation/manipulation/models/bin_small.sdf")

    X_7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, 0), [0.0, 0.02625, 0.12])
    plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa_model_instance),
                     plant.GetFrameByName("bin_base", gripper), X_7G)
    return gripper

In [3]:
def AddFastIiwaDifferentialIK(builder, plant, frame=None):
    params = DifferentialInverseKinematicsParameters(plant.num_positions(),
                                                     plant.num_velocities())
    time_step = plant.time_step()
    q0 = plant.GetPositions(plant.CreateDefaultContext())
    params.set_nominal_joint_position(q0)
    params.set_end_effector_angular_speed_limit(10)
    params.set_end_effector_translational_velocity_limits([-10, -10, -10],
                                                          [10, 10, 10])
    if plant.num_positions() == 3:  # planar iiwa
        iiwa14_velocity_limits = np.array([1.4, 1.3, 2.3])
        params.set_joint_velocity_limits(
            (-iiwa14_velocity_limits, iiwa14_velocity_limits))
        # These constants are in body frame
        assert (
            frame.name() == "iiwa_link_7"
        ), "Still need to generalize the remaining planar diff IK params for different frames"  # noqa
        params.set_end_effector_velocity_flag(
            [True, False, False, True, False, True])
    else:
        iiwa14_velocity_limits = np.ones(7) * 10.0 #np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
        params.set_joint_velocity_limits(
            (-iiwa14_velocity_limits, iiwa14_velocity_limits))
        params.set_joint_centering_gain(10 * np.eye(7))
    if frame is None:
        frame = plant.GetFrameByName("body")
    differential_ik = builder.AddSystem(
        DifferentialInverseKinematicsIntegrator(
            plant,
            frame,
            time_step,
            params,
            log_only_when_result_state_changes=True))
    return differential_ik

In [4]:
def compute_sphere(a, b, c, d):
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(3, 'x')
    prog.AddLinearEqualityConstraint((x - (a + b) / 2).dot(b - a), 0)
    prog.AddLinearEqualityConstraint((x - (c + b) / 2).dot(b - c), 0)
    prog.AddLinearEqualityConstraint((x - (c + d) / 2).dot(d - c), 0)
    res = Solve(prog)
    if res.is_success:
        center = res.GetSolution()
        radius = np.linalg.norm(center - a)
        return np.array([center[0], center[1], center[2], radius])
    
    return None

def compute_sphere_fixed_r(a, b, c, r = 0.02):
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(3, 'x')
    prog.AddConstraint((x - a).dot(x - a) == r*r)
    prog.AddConstraint((x - b).dot(x - b) == r*r)
    prog.AddConstraint((x - c).dot(x - c) == r*r)
    res = Solve(prog)
    if res.is_success:
        center = res.GetSolution()
        radius = np.linalg.norm(center - a)
        return np.array([center[0], center[1], center[2], r])
    
    return None

In [18]:
def icp_find_sphere(point_cloud, num_iters = 500, tol = 0.001):
    best_sol, num_inliers = None, 0
    pts = point_cloud.xyzs()
    pts = pts[:, pts[0, :] != np.inf]
    pts += np.random.randn(*pts.shape) * CAM_NOISE
    if pts.shape[1] < 30:
        return None
    
    for _ in range(num_iters):
        inds = np.random.choice(pts.shape[1], size=4)
        new_sol = compute_sphere(pts[:, inds[0]], pts[:, inds[1]], pts[:, inds[2]], pts[:, inds[3]])
        if new_sol is None:
            continue
            
        pt_dists = np.linalg.norm(pts - new_sol[:3, None], axis=0) - new_sol[3]
        new_inliers = np.sum(np.abs(pt_dists) < tol)
        if new_inliers > num_inliers:
            best_sol, num_inliers = new_sol, new_inliers
            
    #print(best_sol, num_inliers, pts.shape[1])
    return best_sol

def ransac_find_sphere_r(point_cloud, num_iters = 500, tol = 0.001):
    best_sol, num_inliers = None, 0
    pts = point_cloud.xyzs()
    pts = pts[:, pts[0, :] != np.inf]
    pts += np.random.randn(*pts.shape) * CAM_NOISE
    if pts.shape[1] < 30:
        return None
    
    for _ in range(num_iters):
        inds = np.random.choice(pts.shape[1], size=3)
        new_sol = compute_sphere_fixed_r(pts[:, inds[0]], pts[:, inds[1]], pts[:, inds[2]])
        if new_sol is None:
            continue
            
        pt_dists = np.linalg.norm(pts - new_sol[:3, None], axis=0) - new_sol[3]
        new_inliers = np.sum(np.abs(pt_dists) < tol)
        if new_inliers > num_inliers:
            best_sol, num_inliers = new_sol, new_inliers
            
    #print(best_sol, num_inliers, pts.shape[1])
    return best_sol

def prog_find_sphere(point_cloud, num_iters = 500, tol = 0.001):
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(3, 'x')
    r = 0.02
    
    pts = point_cloud.xyzs()
    pts = pts[:, pts[0, :] != np.inf]
    pts += np.random.randn(*pts.shape) * CAM_NOISE
    if pts.shape[1] < 30:
        return None
    
    for i in range(pts.shape[1]):
        dist = (pts[:, i] - x).dot(pts[:, i] - x) - r * r
        prog.AddCost(dist * dist)
        
    res = Solve(prog)
    if res.is_success():
        #print("FOUND")
        #print(res.GetSolution())
        return np.concatenate((res.GetSolution(), np.array([r])))
    else:
        return None
    
def prog_find_sphere_iter(point_cloud, num_iters = 500, tol = 0.001):
    pts = point_cloud.xyzs()
    pts = pts[:, pts[0, :] != np.inf]
    pts += np.random.randn(*pts.shape) * CAM_NOISE
    if pts.shape[1] < 30:
        return None
    
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(3, 'x')
    r = 0.02
    
    for i in range(num_iters * 4):
        i = np.random.choice(pts.shape[1])
        j = np.random.choice(pts.shape[1])
        p1 = pts[:, i]
        p2 = pts[:, j]
        dist = (p1 - x).dot(p1 - x) - (p2 - x).dot(p2 - x)
        prog.AddCost(dist * dist / num_iters)
        
    res = Solve(prog)
    if res.is_success():
        #print("FOUND")
        #print(res.GetSolution())
        return np.concatenate((res.GetSolution(), np.array([r])))
    else:
        return None
    
def prog_find_sphere_no_r(point_cloud, num_iters = 500, tol = 0.001):
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(3, 'x')
    r = prog.NewContinuousVariables(1, 'r')[0]
    
    pts = point_cloud.xyzs()
    pts = pts[:, pts[0, :] != np.inf]
    pts += np.random.randn(*pts.shape) * CAM_NOISE
    if pts.shape[1] < 30:
        return None
    
    for i in range(pts.shape[1]):
        dist = (pts[:, i] - x).dot(pts[:, i] - x) - r * r
        prog.AddCost(dist * dist)
        
    res = Solve(prog)
    if res.is_success():
        #print("FOUND")
        #print(res.GetSolution())
        return res.GetSolution()
    else:
        return None
    
CAM_NOISE = 0.004
SPHERE_FINDER = prog_find_sphere
        

In [64]:

class ICPSphereFinder(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        
        model_point_cloud = AbstractValue.Make(PointCloud(0))

        self.DeclareAbstractInputPort("point_cloud", model_point_cloud)
        self.DeclareVectorOutputPort("sphere_pos", 4,
                                     self.CalcOutput)
        
        self.count_ind = self.DeclareDiscreteState(1)

    def CalcOutput(self, context, output):
        if context.get_discrete_state().get_value() < 2:
            cloud = self.get_input_port().Eval(context)
            sphere = SPHERE_FINDER(cloud, 500, 0.003)
            if sphere is not None:
                #print("FOUND", context.get_time())
                output.SetFromVector(sphere)
                context.get_mutable_discrete_state().set_value(context.get_discrete_state().get_value() + 1)
            else:
                output.SetFromVector(np.ones(4) * np.nan)
        else:
            output.SetFromVector(np.ones(4) * np.nan)
            
class TrajectoryEstimator(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        self._g = np.array([0, 0, 9.8067])
        
        self.DeclareVectorInputPort("sphere_position", 4)
        self.DeclareVectorInputPort("time", 1)
        self.DeclareVectorOutputPort("trajectory_estimate", 6, self.CalcOutput)
        
        self.estimate_ind = self.DeclareDiscreteState(6)
        self.prev_pos_ind = self.DeclareDiscreteState(np.ones(4) * np.nan) # xyz then t
        
        self.DeclarePeriodicDiscreteUpdateEvent(TIME_STEP, 0.0, self.Update)
    
    def Update(self, context, state):
        sphere_pos = self.get_input_port(0).Eval(context)[:3]
        time = self.get_input_port(1).Eval(context)[0]
        
        
        cur_est = state.get_mutable_value(self.estimate_ind)
        prev = state.get_mutable_value(self.prev_pos_ind)
        prev_pos = prev[:3]
        prev_time = prev[3]
        
        
        if not np.any(np.isnan(sphere_pos)) and not np.any(np.isnan(prev_pos)):
            
            # correct for lag
            time_diff = time - prev_time
            aimed_pos = sphere_pos + self._g / 2 * time_diff * time_diff
            old_vel = (aimed_pos - prev_pos) / time_diff
            sphere_vel = old_vel - self._g * time_diff
            
            start_pos = sphere_pos + time * (-sphere_vel) - self._g / 2 * time * time
            start_vel = sphere_vel + self._g * time
            
            #print("Predicted trajectory:", start_pos, start_vel)

            state_est = np.zeros(6)
            state_est[:3] = start_vel
            state_est[3:] = start_pos
            
            if np.linalg.norm(cur_est) < 0.01:
                cur_est = state_est
            else:
                cur_est = cur_est * 0.7 + state_est * 0.3
                
        state.set_value(self.estimate_ind, cur_est)
        if not np.any(np.isnan(sphere_pos)):
            state.set_value(self.prev_pos_ind, np.array([sphere_pos[0], sphere_pos[1], sphere_pos[2], time]))
        #print(sphere_pos, sphere_vel, cur_est, time)
            
    
    def CalcOutput(self, context, output):
        output.SetFromVector(context.get_discrete_state(self.estimate_ind).get_value())
        
class CatchPlanner(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._g = np.array([0, 0, 9.8067])
        self._plant = plant
        self._bin_body_ind = plant.GetBodyByName("bin_base").index()
        self.DeclareVectorInputPort("estimated_trajectory", 6)
        self.DeclareAbstractInputPort("body_poses", AbstractValue.Make([RigidTransform()]))
        self.int_id = self.DeclareDiscreteState(7)
        self.DeclareVectorOutputPort("interception_data", 6, self.GetOutput)
        self.DeclareVectorOutputPort("interception_time", 1, self.GetTime)
        
        self.DeclarePeriodicUnrestrictedUpdateEvent(TIME_STEP, 0.0, self.Update)
        
        
    def Update(self, context, state):
        traj = self.get_input_port(0).Eval(context)
        traj_vel = traj[:3]
        traj_pos = traj[3:]
        
        if np.linalg.norm(traj) < 0.01:
            state.get_mutable_discrete_state().set_value(np.zeros(7))
        elif not np.all(state.get_discrete_state().get_value() == 0):
            return
        else:
            body_poses = self.get_input_port(1).Eval(context)
            bin_pos = body_poses[self._bin_body_ind].translation()
            prog = MathematicalProgram()
            t = prog.NewContinuousVariables(1, 'x')[0]
            x = traj_pos + t * traj_vel - self._g / 2 * t * t
            prog.AddCost((x - bin_pos).dot(x - bin_pos))
            prog.AddConstraint(t >= 0)
            #prog.AddConstraint(x[:2].dot(x[:2]) + 0.005 * x[2] * x[2] >= 0.2)
            #prog.AddConstraint(x[0] >= 0)
            
            res = Solve(prog)
            t_int = res.GetSolution()
            x_int = traj_pos + t_int * traj_vel - self._g / 2 * t_int * t_int
            v_int = traj_vel - self._g * t_int
            #print("Intercept at time", t_int[0], x_int)
            state.get_mutable_discrete_state().set_value(np.concatenate((x_int, -v_int, 1 * (t_int - 0.1))))
            
    def GetOutput(self, context, output):
        output.set_value(context.get_discrete_state().get_value()[:6])
        
    def GetTime(self, context, output):
        output.set_value(context.get_discrete_state().get_value()[6:])
        
class CatchIKcontroller(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._g = np.array([0, 0, 9.8067])
        self.DeclareVectorInputPort("interception_data", 6)
        self.DeclareVectorInputPort("iiwa_position", 7)
        self.DeclareVectorOutputPort("catch_position", 7, self.GetOutput)
        
        self.cache_id = self.DeclareDiscreteState(7)
        
    def GetOutput(self, context, output):
        cached_output = context.get_discrete_state(self.cache_id).get_value()
        int_data = self.get_input_port(0).Eval(context)
        int_pos, int_ori = int_data[:3], int_data[3:]
        
        iiwa_position = self.get_input_port(1).Eval(context)
        
        
        
        if np.all(cached_output == 0):
            if np.all(int_data == 0):
                output.set_value(iiwa_position)
            else:
                ik = InverseKinematics(self._plant)
                bin_frame = self._plant.GetFrameByName('bin_base')
                
                ik.AddPositionConstraint(
                    bin_frame, [0, 0, 0.01], self._plant.world_frame(),
                    int_pos, int_pos)
                
                if int_ori[2] > -5:
                    ik.AddAngleBetweenVectorsConstraint(
                        bin_frame, [0, 0, 1], self._plant.world_frame(), int_ori, 0, 0)
                else:
                    ik.AddAngleBetweenVectorsConstraint(
                        bin_frame, [0, 0, 1], self._plant.world_frame(), [0, 0, 1], np.pi / 2, np.pi / 2)
                    
                    ik.AddAngleBetweenVectorsCost(
                        bin_frame, [0, 0, 1], self._plant.world_frame(), int_ori, 1)
                
                prog = ik.get_mutable_prog()
                q = ik.q()
                #prog.AddQuadraticErrorCost(np.identity(7), iiwa_position, q[:7])
                prog.SetInitialGuess(q[:7], iiwa_position)
                result = Solve(ik.prog())
                
                if result.is_success():
                    sol = result.GetSolution()
                    context.get_mutable_discrete_state(self.cache_id).set_value(sol[:7])
                    output.set_value(sol[:7])
                else:
                    print("IK Failure!")
                    context.get_mutable_discrete_state(self.cache_id).set_value(iiwa_position)
                    output.set_value(iiwa_position)
                    
        else:
            output.set_value(cached_output)
            
class CatchCompensator(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._bin_body_ind = plant.GetBodyByName("bin_base").index()
        self._g = np.array([0, 0, 9.8067])
        self.DeclareAbstractInputPort("body_poses", AbstractValue.Make([RigidTransform()]))
        self.DeclareVectorInputPort("catch_time", 1)
        self.DeclareVectorInputPort("current_time", 1)
        self.DeclareVectorInputPort("interception_data", 6)
        
        self.DeclareAbstractOutputPort("target_pose", lambda : AbstractValue.Make(RigidTransform()), self.GetOutput)
        
        self.traj_id = self.DeclareAbstractState(AbstractValue.Make(PiecewisePose()))
        
    def GetOutput(self, context, output):
        traj = context.get_abstract_state(self.traj_id).get_value()
        
        body_poses = self.get_input_port(0).Eval(context)
        bin_pose = body_poses[self._bin_body_ind]
        
        catch_time = self.get_input_port(1).Eval(context)
        cur_time = self.get_input_port(2).Eval(context)
        int_data = self.get_input_port(3).Eval(context)
        int_pos, int_ori = int_data[:3], int_data[3:]
        int_pos -= np.array([0, 0, 0.00])
        int_ori = int_ori / np.linalg.norm(int_ori)
        
        if cur_time > catch_time + 0.2:
            output.set_value(bin_pose)
        
        try:
            output.set_value(traj.GetPose(cur_time))
        except:
            if catch_time == 0 or cur_time < catch_time:
                output.set_value(RigidTransform())
            else:
                #print("switch")
                breaks = [catch_time, catch_time + 0.08, catch_time + 0.16]
                samples_pos = np.array([bin_pose.translation(), bin_pose.translation() - int_ori * 0.05,
                                        bin_pose.translation() - int_ori * 0.1]).T
                #print(samples_pos)
                
                cur_z = (bin_pose.rotation().matrix() @ np.array([[0., 0., 1.]]).T).squeeze()
                axis = np.cross(cur_z, np.array([0, 0, 1]))
                axis = axis / np.linalg.norm(axis)
                target_rot = RotationMatrix(AngleAxis(np.pi / 4, axis)) @ bin_pose.rotation() 
                samples_quat = [bin_pose.rotation(), bin_pose.rotation(), target_rot]
                
                pos_traj = PiecewisePolynomial.FirstOrderHold(breaks, samples_pos)
                quat_traj = PiecewiseQuaternionSlerp(breaks, samples_quat)
                pose_traj = PiecewisePose(pos_traj, quat_traj)
                context.get_mutable_abstract_state(self.traj_id).set_value(pose_traj)
                output.set_value(pose_traj.GetPose(cur_time))
            
            
class CatchControlSwitch(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("catch_time", 1)
        self.DeclareVectorInputPort("current_time", 1)
        self.DeclareVectorInputPort("ik_position", 7)
        self.DeclareVectorInputPort("compensator_position", 7)
        
        self.DeclareVectorOutputPort("position", 7, self.GetOutput)
        self.DeclareAbstractOutputPort("reset_diffik", lambda : AbstractValue.Make(False), self.ResetDiffIK)
        
    def GetOutput(self, context, output):
        catch_time = self.get_input_port(0).Eval(context)
        
        cur_time = self.get_input_port(1).Eval(context)
        
        if catch_time == 0:
            catch_time = cur_time + 10
            
        correct_port = 2 if cur_time < catch_time else 3
        output.set_value(self.get_input_port(correct_port).Eval(context))
        
    def ResetDiffIK(self, context, output):
        catch_time = self.get_input_port(0).Eval(context)
        cur_time = self.get_input_port(1).Eval(context)
        output.set_value(cur_time < catch_time)

# class CatchIKPlanner(LeafSystem):
#     def __init__(self, plant):
        
#         LeafSystem.__init__(self)
#         self._plant = plant
#         self._g = np.array([0, 0, 9.81])
                
#         self.DeclareVectorInputPort("estimated_trajectory", 6)
#         self.DeclareVectorInputPort("iiwa_position", 7)
#         self.DeclareVectorOutputPort("catch_position", 7, self.GetOutput)    
#         self.cache_id = self.DeclareDiscreteState(7)
        
#     def GetOutput(self, context, output):
#         cached_output = context.get_discrete_state(self.cache_id).get_value()
#         print(cached_output)
#         traj = self.get_input_port(0).Eval(context)
#         traj_vel = traj[:3]
#         traj_pos = traj[3:]
#         iiwa_position = self.get_input_port(1).Eval(context)
        
#         if not np.all(cached_output == 0):
#             output.set_value(cached_output)
#         else:
#             if np.all(traj == 0):
#                 output.set_value(iiwa_position)
#             else:
#                 ik = InverseKinematics(self._plant)
#                 bin_frame = self._plant.GetFrameByName('bin_base')
#                 prog = ik.get_mutable_prog()
#                 q = ik.q()
#                 t = prog.NewContinuousVariables(1, 't')
#                 x = traj_pos + t * traj_vel - self._g / 2 * t * t
                
#                 ik.AddPositionConstraint(
#                     bin_frame, [0, 0, 0], self._plant.world_frame(),
#                     x, x)
                
#                 prog.AddQuadraticErrorCost(np.identity(7), iiwa_position, q[:7])
#                 prog.SetInitialGuess(q[:7], iiwa_position)
#                 result = Solve(ik.prog())
                
#                 if result.is_success():
#                     sol = result.GetSolution()
#                     print(f"intercept at time {sol[-1]}")
#                     context.get_mutable_discrete_state(self.cache_id).set_value(sol[:7])
#                     output.set_value(sol[:7])
#                 else:
#                     print("IK Failure!")
#                     context.get_mutable_discrete_state(self.cache_id).set_value(iiwa_position)
#                     output.set_value(iiwa_position)
                
                
                

In [7]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [68]:
meshcat.Delete()
meshcat.DeleteAddedControls()
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
plant.set_discrete_contact_solver(DiscreteContactSolver.kSap)
parser = Parser(plant, scene_graph)
AddPackagePaths(parser)

# Note that we parse into both the plant and the scene_graph here.
iiwa_model, cam_model, bin_model = parser.AddModels(
        "../manipulation/manipulation/models/catching_env.dmd.yaml")

projectile = AddShape(plant, Sphere(0.02), "projectile", color=[.4, 1, 1, 1])
proj_body = plant.GetBodyByName("projectile")
plant.SetDefaultFreeBodyPose(proj_body, RigidTransform(RotationMatrix.Identity(), [-2, 0, 0]))


plant.Finalize()
AddRgbdSensors(builder, plant, scene_graph)

# Adds the MeshcatVisualizer and wires it to the SceneGraph.
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

num_iiwa_positions = plant.num_positions(iiwa_model)
model_instance_name = "iiwa"
model_instance = iiwa_model

# I need a PassThrough system so that I can export the input port.
iiwa_position = builder.AddSystem(PassThrough(num_iiwa_positions))
# builder.ExportInput(iiwa_position.get_input_port(),
#                     model_instance_name + "_position")
builder.ExportOutput(iiwa_position.get_output_port(),
                     model_instance_name + "_position_commanded")

# Export the iiwa "state" outputs.
demux = builder.AddSystem(
    Demultiplexer(2 * num_iiwa_positions, num_iiwa_positions))
builder.Connect(plant.get_state_output_port(model_instance),
                demux.get_input_port())
builder.ExportOutput(demux.get_output_port(0),
                     model_instance_name + "_position_measured")
builder.ExportOutput(demux.get_output_port(1),
                     model_instance_name + "_velocity_estimated")
builder.ExportOutput(plant.get_state_output_port(model_instance),
                     model_instance_name + "_state_estimated")

# Make the plant for the iiwa controller to use.
controller_plant = MultibodyPlant(time_step=0.002)
controller_plant.set_discrete_contact_solver(DiscreteContactSolver.kSap)
controller_iiwa = AddIiwa(controller_plant)
AddBin(controller_plant, controller_iiwa)
controller_plant.Finalize()

# Add the iiwa controller
iiwa_controller = builder.AddSystem(
    InverseDynamicsController(controller_plant,
                              kp=[10] * num_iiwa_positions,
                              ki=[2] * num_iiwa_positions,
                              kd=[10] * num_iiwa_positions,
                              has_reference_acceleration=False))
iiwa_controller.set_name(model_instance_name + "_controller")
builder.Connect(plant.get_state_output_port(model_instance),
                iiwa_controller.get_input_port_estimated_state())

# Add in the feed-forward torque
adder = builder.AddSystem(Adder(2, num_iiwa_positions))
builder.Connect(iiwa_controller.get_output_port_control(),
                adder.get_input_port(0))
# Use a PassThrough to make the port optional (it will provide zero
# values if not connected).
torque_passthrough = builder.AddSystem(
    PassThrough([0] * num_iiwa_positions))
builder.Connect(torque_passthrough.get_output_port(),
                adder.get_input_port(1))
builder.ExportInput(torque_passthrough.get_input_port(),
                    model_instance_name + "_feedforward_torque")
builder.Connect(adder.get_output_port(),
                plant.get_actuation_input_port(model_instance))

# Add discrete derivative to command velocities.
desired_state_from_position = builder.AddSystem(
    StateInterpolatorWithDiscreteDerivative(
        num_iiwa_positions,
        0.002,
        suppress_initial_transient=True))
desired_state_from_position.set_name(
    model_instance_name + "_desired_state_from_position")
builder.Connect(desired_state_from_position.get_output_port(),
                iiwa_controller.get_input_port_desired_state())
builder.Connect(iiwa_position.get_output_port(),
                desired_state_from_position.get_input_port())

builder.ExportOutput(adder.get_output_port(),
                     model_instance_name + "_torque_commanded")
builder.ExportOutput(adder.get_output_port(),
                     model_instance_name + "_torque_measured")

builder.ExportOutput(
    plant.get_generalized_contact_forces_output_port(
        model_instance), model_instance_name + "_torque_external")

# # (optional) Add Meshcat Camera View Visualizer
# point_cloud_visualizer = builder.AddSystem(
#     MeshcatPointCloudVisualizer(meshcat, "cloud"))
# builder.Connect(builder.GetSystems()[3].point_cloud_output_port(),
#                 point_cloud_visualizer.cloud_input_port())

differential_ik = AddFastIiwaDifferentialIK(builder, 
                                            controller_plant, 
                                            controller_plant.GetFrameByName('bin_base'))
builder.Connect(plant.get_state_output_port(model_instance),
                differential_ik.get_input_port(1))

# Add ICP Sphere Finder
sphere_finder = builder.AddSystem(
    ICPSphereFinder())

builder.Connect(builder.GetSystems()[3].point_cloud_output_port(),
                sphere_finder.get_input_port())

# Add integrator to get time
int_source = builder.AddSystem(
    ConstantVectorSource([1.0]))
timer = builder.AddSystem(
    Integrator(1))
builder.Connect(int_source.get_output_port(),
               timer.get_input_port())

# Add sphere trajectory estimator
sphere_traj_est = builder.AddSystem(
    TrajectoryEstimator())
builder.Connect(sphere_finder.get_output_port(),
                sphere_traj_est.get_input_port(0))
builder.Connect(timer.get_output_port(),
                sphere_traj_est.get_input_port(1))

# Add catch planner
catch_planner = builder.AddSystem(
    CatchPlanner(plant))
builder.Connect(sphere_traj_est.get_output_port(),
                catch_planner.get_input_port(0))
builder.Connect(plant.get_body_poses_output_port(),
                catch_planner.get_input_port(1))

# builder.Connect(catch_planner.get_output_port(),
#                 differential_ik.get_input_port(0))

# Add catch controllers and switch
catch_switch = builder.AddSystem(
    CatchControlSwitch())
catch_controller = builder.AddSystem(
    CatchIKcontroller(controller_plant))
catch_compensator = builder.AddSystem(
    CatchCompensator(plant))

builder.Connect(catch_planner.get_output_port(0),
               catch_controller.get_input_port(0))
builder.Connect(demux.get_output_port(0),
               catch_controller.get_input_port(1))

builder.Connect(plant.get_body_poses_output_port(),
               catch_compensator.get_input_port(0))
builder.Connect(catch_planner.get_output_port(1),
               catch_compensator.get_input_port(1))
builder.Connect(timer.get_output_port(),
               catch_compensator.get_input_port(2))
builder.Connect(catch_planner.get_output_port(0),
               catch_compensator.get_input_port(3))
builder.Connect(catch_compensator.get_output_port(),
                differential_ik.get_input_port(0))

builder.Connect(catch_planner.get_output_port(1),
               catch_switch.get_input_port(0))
builder.Connect(timer.get_output_port(),
               catch_switch.get_input_port(1))
builder.Connect(catch_controller.get_output_port(),
               catch_switch.get_input_port(2))
builder.Connect(differential_ik.get_output_port(),
               catch_switch.get_input_port(3))

builder.Connect(catch_switch.get_output_port(0),
               iiwa_position.get_input_port())
builder.Connect(catch_switch.get_output_port(1),
                differential_ik.get_input_port(2))

# combined catch planner
# catch_ik_planner = builder.AddSystem(
#     CatchIKPlanner(controller_plant))

# builder.Connect(sphere_traj_est.get_output_port(),
#                catch_ik_planner.get_input_port(0))
# builder.Connect(demux.get_output_port(0),
#                catch_ik_planner.get_input_port(1))
# builder.Connect(catch_ik_planner.get_output_port(),
#                iiwa_position.get_input_port())

diagram = builder.Build()

In [100]:
SPHERE_FINDER = prog_find_sphere_iter
CAM_NOISE = 0.0
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
sphere_context = sphere_finder.GetMyMutableContextFromRoot(context)
diagram.Publish(context)

p_init = [-3, 0.25, -1]
v_init = [7, -0.1, 6.1]
plant.SetFreeBodyPose(plant_context, proj_body, RigidTransform(RotationMatrix.Identity(), p_init))
plant.SetFreeBodySpatialVelocity(proj_body, SpatialVelocity([0, 0, 0], v_init), plant_context)
print(plant.GetPositions(plant_context))
print(plant.GetPositions(plant_context, iiwa_model))
# iiwa_position.get_input_port().FixValue(position_context, [1, 1, 1, 1, 1, 1, 1])
# differential_ik.get_input_port(0).FixValue(diff_ik_context, RigidTransform(RotationMatrix.Identity(), [0, 0, 0]))
# differential_ik.GetInputPort("use_robot_state").FixValue(diff_ik_context, False)

simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)

visualizer.StartRecording()
simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
visualizer.StopRecording()
visualizer.PublishRecording()

[ 0.    0.1   0.   -1.2   0.    1.6   0.    1.    0.    0.    0.   -3.
  0.25 -1.  ]
[ 0.   0.1  0.  -1.2  0.   1.6  0. ]


In [94]:
# Automatic sampling of a trajectory

def gen_traj():
    # First sample a starting point visible to the camera
    p_init = np.array([np.random.uniform(-3.5, -2), np.random.uniform(-0.5, 0.5), np.random.uniform(-1, 0)])

    # Then, sample a potential interception point
    theta_int = np.random.uniform(2 * np.pi)
    p_int = np.array([0.5 + 0.5*np.sin(theta_int), 0.3 + 0.75 * np.cos(theta_int), np.random.uniform(-0.25, 0.75)])
    t_int = np.random.uniform(0.5, 1)
    print(p_int, t_int)

    # Calculate initial velocity
    g = np.array([0, 0, 9.8067])
    p_adj = p_int + 0.5 * g * t_int * t_int
    v_init = (p_adj - p_init) / t_int

    return p_init, v_init


# ================================================================================

while True:
    SPHERE_FINDER = prog_find_sphere_iter
    CAM_NOISE = 0.0
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    sphere_context = sphere_finder.GetMyMutableContextFromRoot(context)
    diagram.Publish(context)

    p_init, v_init = gen_traj()
    plant.SetFreeBodyPose(plant_context, proj_body, RigidTransform(RotationMatrix.Identity(), p_init))
    plant.SetFreeBodySpatialVelocity(proj_body, SpatialVelocity([0, 0, 0], v_init), plant_context)

    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)

    visualizer.StartRecording()
    simulator.AdvanceTo(1.5 if running_as_notebook else 0.1)
    visualizer.StopRecording()
    visualizer.PublishRecording()
    break

[0.99738464 0.22338956 0.33170391] 0.7702226391983894


In [83]:
traj_results  = dict()

In [36]:
# Perception Error Tests
CAM_NOISE = 0.00
SPHERE_FINDER = ransac_find_sphere_r


def run_perception_test(cam_noise, sphere_func, num_trials):
    global CAM_NOISE
    global SPHERE_FINDER
    CAM_NOISE = cam_noise
    SPHERE_FINDER = sphere_func
    i = 0
    logs = []
    while i < num_trials:
        p_init = np.array([np.random.uniform(-3.5, -2), np.random.uniform(-0.5, 0.5), np.random.uniform(-1, 0)])
        context = diagram.CreateDefaultContext()
        plant_context = plant.GetMyMutableContextFromRoot(context)
        plant.SetFreeBodyPose(plant_context, proj_body, RigidTransform(RotationMatrix.Identity(), p_init))
        sphere_context = sphere_finder.GetMyMutableContextFromRoot(context)
        found_sphere = sphere_finder.get_output_port().Eval(sphere_context)[:3]
        if not np.isnan(found_sphere[0]):
            if i % 10 == 0:
                print(f"FOUND {i}")
            i += 1
            logs.append(100 * np.linalg.norm(found_sphere - p_init))
        else:
            print("MISSED")
    return logs

test_noises = [0.0, 0.005, 0.01, 0.05]

perception_results['ransac_unknown_r'] = dict()
for noise in test_noises:
    perception_results['ransac_unknown_r'][noise] = run_perception_test(noise, icp_find_sphere, 200)

MISSED
MISSED
MISSED
FOUND 0
MISSED
FOUND 10
MISSED
FOUND 20
MISSED
MISSED
MISSED
FOUND 30
MISSED
FOUND 40
MISSED
MISSED
FOUND 50
FOUND 60
MISSED
FOUND 70
FOUND 80
MISSED
MISSED
FOUND 90
MISSED
FOUND 100
MISSED
FOUND 110
FOUND 120
MISSED
FOUND 130
MISSED
MISSED
FOUND 140
MISSED
MISSED
MISSED
FOUND 150
FOUND 160
FOUND 170
MISSED
FOUND 180
MISSED
FOUND 190
MISSED
FOUND 0
MISSED
FOUND 10
FOUND 20
MISSED
MISSED
MISSED
FOUND 30
MISSED
MISSED
MISSED
FOUND 40
FOUND 50
MISSED
MISSED
FOUND 60
MISSED
FOUND 70
MISSED
FOUND 80
MISSED
FOUND 90
FOUND 100
MISSED
FOUND 110
MISSED
FOUND 120
FOUND 130
FOUND 140
MISSED
FOUND 150
MISSED
FOUND 160
MISSED
FOUND 170
FOUND 180
FOUND 190
MISSED
FOUND 0
MISSED
MISSED
MISSED
MISSED
FOUND 10
MISSED
MISSED
FOUND 20
MISSED
FOUND 30
MISSED
FOUND 40
MISSED
FOUND 50
FOUND 60
FOUND 70
FOUND 80
FOUND 90
FOUND 100
MISSED
FOUND 110
MISSED
FOUND 120
FOUND 130
FOUND 140
MISSED
MISSED
MISSED
FOUND 150
MISSED
FOUND 160
FOUND 170
FOUND 180
MISSED
FOUND 190
FOUND 0
MISSED
FOUND

In [115]:
# traj error
SPHERE_FINDER = prog_find_sphere_iter
CAM_NOISE = 0.005
def run_traj_trial():
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    diagram.Publish(context)

    p_init, v_init = gen_traj()
    plant.SetFreeBodyPose(plant_context, proj_body, RigidTransform(RotationMatrix.Identity(), p_init))
    plant.SetFreeBodySpatialVelocity(proj_body, SpatialVelocity([0, 0, 0], v_init), plant_context)

    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.2)
    
    traj_context = sphere_traj_est.GetMyContextFromRoot(context)
    planner_context = catch_planner.GetMyContextFromRoot(context)
    catch_pt = catch_planner.get_output_port(0).Eval(planner_context)[:3]
    catch_time = catch_planner.get_output_port(1).Eval(planner_context) + 0.1
    g = np.array([0, 0, 9.8067])
    real_pt = p_init + v_init * catch_time - 1/2 * g * catch_time * catch_time
    if np.all(catch_pt == 0):
        return None
    
    error = np.linalg.norm(catch_pt - real_pt)
    
    return error

def run_traj_test(noise, sphere_func, num_trials):
    global CAM_NOISE
    global SPHERE_FINDER
    
    CAM_NOISE = noise
    SPHERE_FINDER = sphere_func
    
    i = 0
    logs = []
    
    while i < num_trials:
        error = run_traj_trial()
        if error is not None:
            logs.append(error)
            i += 1
            print(i)
        
    return logs
        

    
#traj_results['prog_unknown_r'] = dict()
test_noises = [0.01]
for noise in test_noises:
    traj_results['ransac_unknown_r'][noise] = run_traj_test(noise, icp_find_sphere, 50)

1
2
3
IK Failure!
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
IK Failure!
19
20
21
22
23
24
IK Failure!
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
IK Failure!
47
IK Failure!
48
49
50


In [129]:
test_noises = [0.0, 0.005, 0.01]
for noise in test_noises:
    test_arr = np.minimum(100 * np.array(traj_results['ransac_unknown_r'][noise]), 50)
    print(f"${np.median(test_arr):.3f} \pm {scipy.stats.sem(test_arr):.3f}$")
print("+++++++++++++++++++++")
for noise in test_noises:
    test_arr = np.minimum(100 * np.array(traj_results['prog_unknown_r'][noise]), 50)
    print(f"${np.mean(test_arr):.3f} \pm {scipy.stats.sem(test_arr):.3f}$")

$0.222 \pm 0.021$
$16.379 \pm 1.724$
$33.516 \pm 2.177$
+++++++++++++++++++++
$0.186 \pm 0.015$
$4.591 \pm 0.288$
$8.918 \pm 0.513$


In [152]:
# full pipeline results
test_arr = 100 * np.array([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 
0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,  0, 1, 1, 1, 1,  1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,  1]
) # compensation, kp = kd = 50
print(f"${np.mean(test_arr):.2f} \pm {scipy.stats.sem(test_arr):.2f}$")

test_arr = 100 * np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0])
print(f"${np.mean(test_arr):.2f} \pm {scipy.stats.sem(test_arr):.2f}$")

$79.41 \pm 4.94$
$42.31 \pm 5.63$


In [ ]:
# Make Figures

meshcat.SetObject("sphere1", Sphere(0.02), Rgba(.4, 1, 1, 1))
sphere_pos = np.array(p_init) + 0.0936 * np.array(v_init) - 0.0936 * 0.0936 * np.array([0, 0, 4.905])
meshcat.SetTransform("sphere1", RigidTransform(RotationMatrix.Identity(), sphere_pos))

meshcat.SetObject("sphere2", Sphere(0.02), Rgba(.4, 1, 1, 1))
sphere_pos = np.array(p_init)
meshcat.SetTransform("sphere2", RigidTransform(RotationMatrix.Identity(), sphere_pos))

meshcat.SetObject("int_sphere", Sphere(0.02), Rgba(.4, .4, 1, 1))
int_pos = np.array([0.46166529, -0.49930749, 0.79919653])
meshcat.SetTransform("int_sphere", RigidTransform(RotationMatrix.Identity(), int_pos))

traj_step = 0.01
for i in np.arange(0, 1.5, traj_step):
    sphere_pos = np.array(p_init) + i * np.array(v_init) - i * i * np.array([0, 0, 4.905])
    meshcat.SetObject(f"traj/sphere{i}", Sphere(0.01), Rgba(.9, .5, .5, 1))
    meshcat.SetTransform(f"traj/sphere{i}", RigidTransform(RotationMatrix.Identity(), sphere_pos))